In [4]:
import os
import pandas as pd
import soundfile as sf
from collections import Counter
import scipy.signal as signal
import matplotlib.pyplot as plt
import gc
import IPython.display as ipd 

Download the dataset from https://www.kaggle.com/toponowicz/spoken-language-identification

In [6]:
DATA_FOLDER = '../datasets/langid/'
train_path = DATA_FOLDER+'/train/'
test_path = DATA_FOLDER+'/test/'

#para train path
train_labels = []
for filename in os.listdir(train_path):
    train_labels.append(filename[:4]) # [:4] file naming convention, f.e. es_f
test_labels = []
for filename in os.listdir(test_path):
    test_labels.append(filename[:4])

train_files = []
for filename in os.listdir(train_path):
    train_files.append(filename)
test_files = []
for filename in os.listdir(test_path):
    test_files.append(filename)

# Create numeric labels
lb=0
label_d={}
for k in Counter(train_labels).keys():
    label_d[k] = lb
    lb=lb +1


train_data = {'filename':train_files,
       'lang_gender':train_labels}
train_df = pd.DataFrame(train_data)
test_data = {'filename':test_files,
       'lang_gender':test_labels}
test_df = pd.DataFrame(test_data)

In [7]:
test_df.head()

,filename,language
0,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en_f
1,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...,en_m
2,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en_f
3,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de_f
4,de_m_923551d571cc437382d0294dda2dd0aa.fragment...,de_m


In [8]:
# Create numeric labels
lb=0
label_d={}
for k in Counter(train_labels).keys():
    label_d[k] = lb
    lb=lb +1
for x in label_d.items():
    print(x)

('de_f', 0)
('es_f', 1)
('en_m', 2)
('en_f', 3)
('de_m', 4)
('es_m', 5)


In [ ]:
# Create the labeled training and test data
x_train=[]
y_train=[]
for filename in os.listdir(train_path):
    y_train.append(label_d[filename[:4]])
    x_train.append(sf.read(train_path+filename))

x_test=[]
y_test=[]
for filename in os.listdir(test_path):
    y_test.append(label_d[filename[:4]])
    x_test.append(sf.read(test_path+filename))


no_train = len(x_train)
no_test = len(x_test)
no_labels = len(np.unique(x_train))

print('# training samples {}'.format(no_train))
print('# test samples {}'.format(no_test))
print('# uniques labels {}'.format(no_labels))

('de_f', 0)
('es_f', 1)
('en_m', 2)
('en_f', 3)
('de_m', 4)
('es_m', 5)


In [2]:
np.shape(x_train)

NameError: name 'np' is not defined